In [1]:
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
import logging
from datetime import datetime
import sys
import os
import gzip
import csv
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
nfile = 'data/phishing_final.xlsx'
DATA_PATH= 'data/'

In [4]:
emailpool = pd.read_excel(nfile,sheet_name = 'emailpool')
spear_phishing = pd.read_excel(nfile,sheet_name='spear_phishing')
userclassification = pd.read_excel(nfile,sheet_name='userclassification')
rater = pd.read_csv(DATA_PATH+'rater.compiled.csv')
raterids = rater['UserID'].tolist()

In [5]:
nfile = 'data/emails_survey_combined.csv'
con_emails = pd.read_csv(nfile)

In [6]:
import sklearn
surveyfields = ['important', 'action', 'information', 'project',
       'meeting', 'spam', 'deadline', 'other']
surveymatrix = con_emails[surveyfields].fillna(0).values
simmatirx_survey = sklearn.metrics.pairwise.cosine_similarity(surveymatrix,surveymatrix) 

In [7]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('nli-distilroberta-base-v2')#'stsb-roberta-large')


In [8]:
n = simmatirx_survey.shape[0]
allpoints = []
for i in range(n):
    for j in range(n):
        if i>j:
            allpoints.append((i,j))
allpoints = np.array(allpoints)

In [9]:
n = 40000
chosedata = np.random.choice(len(allpoints),n)
trainidx = allpoints[chosedata[:int(n*0.7)]]
devidx = allpoints[chosedata[int(n*0.7):int(n*0.8)]]
testidx = allpoints[chosedata[int(n*0.8):]]

In [1]:
testidx[:5]

NameError: name 'testidx' is not defined

In [11]:
con_emails.loc[2]['cleantext_withpunc']

'Urgent - Amazon Web Services Account Suspended Reference number :& 108 - 4596473 - 8009841 FF ;& Hello ,& ;& Amazon unable confirm important details Amazon Web Services ( AWS ) Account . Your AWS account suspended result .& ;& Please verify account details confirm payment information needed Amazon Web Services Account .& Updating information also ensure protected security attacks . Update Verify Your Payment Method Failure Update Payment Method 02 / 01 / 2019 result complete deletion account .& Account Details Account # 108 - 4596473 - 8009841 ;& If questions please &# 39 hesitate contact customer service . If questions particular email please include reference number seen top email . Please reply e - mail directly .& We hope able assistance .& Amazon IT Team , Conditions Use Privacy Notice 1996 - 2015 , Amazon . com , Inc . affiliates 05 / 10 / 2019'

In [12]:
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses

#model = SentenceTransformer('distilbert-base-nli-mean-tokens')
train_examples = [InputExample(texts=['My first sentence', 'My second sentence'], label=0.8),
    InputExample(texts=['Another pair', 'Unrelated sentence'], label=0.3)]
train_dataset = SentencesDataset(train_examples, model)

In [13]:
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses

#model = SentenceTransformer('distilbert-base-nli-mean-tokens')
#[InputExample(texts=['My first sentence', 'My second sentence'], label=0.8),
#    InputExample(texts=['Another pair', 'Unrelated sentence'], label=0.3)]
#train_dataset = SentencesDataset(train_examples, model)
def prepare_data(indexlist):
    examples = []

    for idx in indexlist:
        temp = InputExample(texts = [con_emails.loc[idx[0]]['cleantext_withpunc'],
                                     con_emails.loc[idx[1]]['cleantext_withpunc']],
                           label = float(simmatirx_survey[idx[0]][idx[1]]))
        examples.append(temp)
    return SentencesDataset(examples,model)
train = prepare_data(trainidx)
valid = prepare_data(devidx)
test = prepare_data(testidx)

In [13]:

train_batch_size = 32
num_epochs = 10
model_save_path = 'data/model1/'

In [15]:
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test, name='sts-test')
test_evaluator(model, output_path=model_save_path)

0.13277553991414254

In [16]:
train_dataloader = DataLoader(train, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)


logging.info("phishing dataset")
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(valid, name='sts-dev')


# Configure the training. We skip evaluation in this example
warmup_steps = math.ceil(len(train_dataloader) * num_epochs  * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))
model.device 

# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=100,
          warmup_steps=warmup_steps,
          output_path=model_save_path)

In [17]:
model = SentenceTransformer(model_save_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test, name='sts-test')

test_evaluator(model, output_path=model_save_path)

0.9882496041422227

In [29]:
' '.join(surveyfields)

'important action information project meeting spam deadline other'